In [2]:
import tensorflow as tf

BATCH_SIZE = 32

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'Testing',
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    shuffle=False                # keep order for matching preds to labels
).map(lambda x,y: (x/255.0, y))

Found 191 files belonging to 2 classes.


In [9]:
import os
import pandas as pd

data_dir = 'Testing'
filepaths = []
labels    = []

for label_name, label_value in [('cancer', 1), ('normal', 0)]:
    folder = os.path.join(data_dir, label_name)
    for fname in os.listdir(folder):
        if fname.lower().endswith(('.png','.jpg','.jpeg')):
            filepaths.append(os.path.join(folder, fname))
            labels.append(label_value)

df = pd.DataFrame({
    'filepath': filepaths,
    'label': labels
})
# (Optional) save to CSV
df.to_csv('external_labels.csv', index=False)


In [10]:
import pandas as pd

df = pd.read_csv('external_labels.csv')

def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels=3)
    # img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32) / 255.0
    return img, label

paths = df['filepath'].values
labels = df['label'].values

test_ds = tf.data.Dataset.from_tensor_slices((paths, labels))
test_ds = test_ds.map(load_and_preprocess).batch(BATCH_SIZE)


In [12]:
import tensorflow as tf

def load_and_preprocess(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    # img = tf.image.resize(img, (224, 224))
    # img = tf.cast(img, tf.float32) / 255.0
    return img, label

paths  = df['filepath'].values
labels = df['label'].values

test_ds = tf.data.Dataset.from_tensor_slices((paths, labels))
test_ds = test_ds.map(load_and_preprocess).batch(BATCH_SIZE)


In [14]:
model = tf.keras.models.load_model('model.h5')

c:\Users\91951\Downloads\Lungs-Cancer-Detection-main\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\91951\Downloads\Lungs-Cancer-Detection-main\.venv\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(224,224,3)),
    # … more convs …
    tf.keras.layers.GlobalAveragePooling2D(),   # or Flatten()
    tf.keras.layers.Dense(1, activation='sigmoid'),
])


c:\Users\91951\Downloads\Lungs-Cancer-Detection-main\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
import tensorflow as tf

IMG_SIZE   = (224, 224)   # or whatever your model expects
BATCH_SIZE = 32

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'Testing',
    labels='inferred',
    label_mode='int',
    image_size=IMG_SIZE,     # <-- this resizes every image
    batch_size=BATCH_SIZE,
    shuffle=False
).map(lambda x, y: (x/255.0, y))


Found 191 files belonging to 2 classes.


In [32]:
# Quick overall accuracy
# loss, acc = model.evaluate(test_ds)
# print(f"Loss: {loss:.4f}, Accuracy: {acc*100:.2f}%")
import sklearn
# Or for full metrics:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

y_true, y_preds, y_prob = [], [], []
for imgs, lbls in test_ds:
    probs = model.predict(imgs)            # shape (batch, 1)
    # Option A: take column 0
    pos_probs = probs[:, 0]

    # Option B: squeeze into (batch,)
    pos_probs = probs.squeeze(axis=1)      # now shape is (batch,)

    # then threshold
    y_pred = (pos_probs >= 0.5).astype(int)            # shape (batch, 1)

    y_true.extend(lbls.numpy().tolist())
    y_preds.extend(y_pred)
    y_prob.extend(probs)

y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_prob = np.array(y_prob)

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall (sensitivity):", recall_score(y_true, y_pred))
print("AUC:", roc_auc_score(y_true, y_prob))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 363ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 385ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 581ms/step


ValueError: Found input variables with inconsistent numbers of samples: [191, 31]

In [33]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

# 1) Initialize as empty Python lists
y_true_list = []
y_pred_list = []
y_prob_list = []

# 2) Loop once over your dataset, extending both lists equally
for imgs, labels in test_ds:
    # Predict cancer probability → shape (batch,1), then squeeze to (batch,)
    probs = model.predict(imgs).squeeze(axis=1)
    # Binary predictions
    preds = (probs >= 0.5).astype(int)

    # Convert labels and preds to Python lists, then extend
    y_true_list.extend(labels.numpy().tolist())
    y_pred_list.extend(preds.tolist())
    y_prob_list.extend(probs.tolist())

# 3) Debug: print lengths to verify match
print(f"Num labels:      {len(y_true_list)}")
print(f"Num predictions: {len(y_pred_list)}")
print(f"Num probabilities: {len(y_prob_list)}")
assert len(y_true_list) == len(y_pred_list) == len(y_prob_list), "Counts don’t match!"

# 4) Convert to NumPy arrays
y_true = np.array(y_true_list)
y_pred = np.array(y_pred_list)
y_prob = np.array(y_prob_list)

# 5) Compute metrics
print("Accuracy:   ", accuracy_score(y_true, y_pred))
print("Precision:  ", precision_score(y_true, y_pred))
print("Recall:     ", recall_score(y_true, y_pred))
print("AUC:        ", roc_auc_score(y_true, y_prob))


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 751ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 412ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 451ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 428ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 350ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 403ms/step
Num labels:      191
Num predictions: 191
Num probabilities: 191
Accuracy:    0.8481675392670157
Precision:   0.0
Recall:      0.0
AUC:         0.17369093231162194


c:\Users\91951\Downloads\Lungs-Cancer-Detection-main\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
